In [1]:
#upload some libraries
import os
import pandas.testing as tm
import h5py
import numpy as np
import pandas as pd
import math
import sys
import subprocess
import statsmodels.api as sm
from scipy import stats
import statsmodels.formula.api as smf
#patsy builtins are for . in transcriptom ID
from patsy.builtins import *
import statistics
import umap.umap_ as umap
from joblib import dump, load
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
from sklearn.decomposition import PCA

! pip install --upgrade tables
import tables

# set paths
basedir = '/data/songy4/tes'
datadir = f'{basedir}/data_folder'
workdir = f'{datadir}/output'

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
#download the renormalized datasets ---- run it only once
#TElocal output files normalized with limma for a different timepoint, weren't adjusted for any covariates
! cp -R /data/LNG/Frank/TE/limma/baseline_GENE_TE_norm_adjusted.cntTable /{datadir}/
! cp -R /data/LNG/Frank/TE/limma/v04_GENE_TE_norm_adjusted.cntTable /{datadir}/
! cp -R /data/LNG/Frank/TE/limma/v06_GENE_TE_norm_adjusted.cntTable /{datadir}/
! cp -R /data/LNG/Frank/TE/limma/v08_GENE_TE_norm_adjusted.cntTable /{datadir}/

#TE location data
! cp -R /data/LNG/Frank/TE/hg38_rmsk_TE.gtf.locInd.locations /{datadir}/

#gene locations
! cp -R /data/LNG/Frank/TE/hg38_gene_annotation.gtf.locations /{datadir}/

#plate information (one of covariates among phenotype (= case vs control), sex, age, plate, expression pcs 1 to 5)
! cp -R /data/LNG/Frank/TE/TElocal_replicates/megaMetaTable.csv /{datadir}/

#genomic data (plink files)
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.bed /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.bim /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.fam /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.log /{datadir}/
! cp -R /data/LNG/Frank/TE/hits_plus_tags_AMPPD_Euros.nosex /{datadir}/

#PD GWAS sum stats
! cp -R /data/LNG/Frank/meta5_sumstats_harmonized_hg19_hg38.csv /{datadir}/

#pheno
! cp -R /data/LNG/Frank/TE/DEMOG_DIAG.csv /{datadir}/

#/data/CARD/PD/summary_stats --> PD GWAS sum stats were based on files from here

Covariate data

In [6]:
#open pheno file megaMetaTable.csv
covariate = pd.read_csv(f"{datadir}/megaMetaTable.csv", index_col=False)

print('shape of covariate data:', covariate.shape)
print(covariate.head())

covariate.describe()

/usr/local/Anaconda/envs/py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,9,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


shape of covariate data: (4891, 138)
  HudAlphaSampleName  Description Alternate MRN Specimen Bar Code  \
0       5104-SL-0001       373111   ST-00009911        0000373111   
1       5104-SL-0002  PP0016-0526   ST-00000049       PP0016-0526   
2       5104-SL-0003      3125874   ST-00001277        0003125874   
3       5104-SL-0004      3179818   ST-00020262        0003179818   
4       5104-SL-0005      3164601   ST-00021309        0003164601   

   Specimen Type  Quantity (ug)  Concentration (ng/ul)  Concentration Flag  \
0  Nucleic Acids            1.0                 115.90                   1   
1  Nucleic Acids            1.0                  80.07                   1   
2  Nucleic Acids            1.0                 219.70                   1   
3  Nucleic Acids            1.0                 100.06                   1   
4  Nucleic Acids            1.0                 124.40                   1   

  Submitted Volume (ul) Normalization Volume (30ng/ul)  ...  \
0           8.62

,Quantity (ug),Concentration (ng/ul),Concentration Flag,260/280 Ratio,260/230 Ratio,RIN Value,POSITION,Plate,PATNO,SITE,...,PCT_R1_TRANSCRIPT_STRAND_READS,MEDIAN_5PRIME_BIAS,PCT_USABLE_BASES,pct_globin,pct_rRNA,genes detected at count > 0,genes detected at count > 10,genes detected at count > 100,genes detected at count > 1000,genes detected at count > 10000
count,4891.000000,4861.000000,4891.000000,4784.000000,4781.000000,4837.000000,4891.000000,4891.000000,4841.000000,4782.000000,...,4878.000000,4878.000000,4878.000000,4879.000000,4879.000000,4877.000000,4877.000000,4877.000000,4877.000000,4877.000000
mean,1.022042,82.121765,0.939890,2.095542,1.309453,7.702791,45.749744,106.505009,20481.995042,124.598703,...,5.209730,0.995533,39.153704,0.009166,0.029934,36876.692434,21645.114210,12340.213656,4696.244002,354.919008
std,0.105439,47.209438,0.237716,0.445532,5.070301,1.646776,27.042168,99.314465,22379.247093,119.956888,...,6.655015,0.072550,9.050471,0.214597,0.618656,8083.571934,4620.730462,1252.296439,1169.621370,180.757908
min,1.000000,4.600000,0.000000,0.240000,-267.800000,0.000000,1.000000,1.000000,1009.000000,1.000000,...,1.050000,0.033756,0.028400,0.000135,0.000250,597.000000,14.000000,4.000000,0.000000,0.000000
25%,1.000000,50.000000,1.000000,2.050000,0.790000,7.200000,22.000000,13.000000,3422.000000,28.000000,...,2.052500,0.964074,35.539100,0.001585,0.002980,30951.000000,19401.000000,11882.000000,4152.000000,237.000000
50%,1.000000,72.200000,1.000000,2.090000,1.360000,8.100000,45.000000,27.000000,3905.000000,73.000000,...,2.665150,1.003158,40.182800,0.002378,0.005781,33340.000000,20189.000000,12428.000000,4823.000000,337.000000
75%,1.000000,103.900000,1.000000,2.130000,1.750000,8.800000,69.000000,212.000000,41432.000000,289.000000,...,5.505450,1.039712,44.599175,0.003741,0.012073,41407.000000,21470.000000,12916.000000,5412.000000,451.000000
max,2.000000,701.100000,1.000000,26.820000,185.000000,10.000000,96.000000,225.000000,92834.000000,335.000000,...,95.589300,1.224844,68.048300,13.544165,38.354647,56464.000000,49526.000000,21136.000000,8145.000000,1438.000000


In [7]:
#check the datatype for column 8 to 12
for col in covariate.columns[8:13]:
    print(str(col) + ': ', type(covariate[col].iat[0]))

#check header
print('Column names: \n',covariate.columns.tolist())

Submitted Volume (ul):  <class 'str'>
Normalization Volume (30ng/ul):  <class 'str'>
Total Volume (ul):  <class 'str'>
Input RNASeq (ng):  <class 'str'>
Input miRNA (ng):  <class 'str'>
Column names: 
 ['HudAlphaSampleName', 'Description', 'Alternate MRN', 'Specimen Bar Code', 'Specimen Type', 'Quantity (ug)', 'Concentration (ng/ul)', 'Concentration Flag', 'Submitted Volume (ul)', 'Normalization Volume (30ng/ul)', 'Total Volume (ul)', 'Input RNASeq (ng)', 'Input miRNA (ng)', '260/280 Ratio', '260/230 Ratio', 'RIN Value', 'BOX', 'POSITION', 'Plate', 'Phase', 'PoolAssign', 'HudAlphaLibraryID', 'sampleid', 'DateAnalyzed', 'PATNO', 'SEX', 'DIAGNOSIS', 'CLINICAL_EVENT', 'SITE', 'CNO', 'QCflagIR3', 'VISIT', 'PATNO_VISIT', 'VISIT_POST_MED_USE', 'VISIT_PRIOR_MED_USE', 'months_between_collection_firstMedUse', 'hours_between_PDdose_collection', 'age_at_consent', 'age_at_diagnosis', 'time_since_diagnosis', 'ageBin', 'Disease_Status', 'Disease_Status_byVisit', 'Study_Arm', 'Most_likely_primary_dia

In [8]:
#pick the columns needed
covariate_df = covariate[['Plate','PATNO','SEX', 'DIAGNOSIS', 'age_at_consent', 'age_at_diagnosis', 'Hispanic_or_Latino','AmericanIndian_or_AlaskaNative','Asian','Black_or_AfricanAmerican','Hawaiian_or_OtherPacificIslander','White','Race_not_specified']]

#check the unique value in white column
print('Unique values in White column: \n', covariate_df.White.unique())

#check nan numbers in each column 
print('Null value in each column: \n', covariate_df.isna().sum())

#check how many each value in white column
print('Value counts in White column: \n',covariate_df.White.value_counts())

#drop rows where NaN
covariate_df = covariate_df[covariate_df.White.notna()]
#drop rows if not Yes
cov_white = covariate_df[covariate_df.White.str.contains("Yes")]

#drop some columns 
cov_df = cov_white[['PATNO', 'SEX', 'DIAGNOSIS', 'age_at_consent', 'White']]
print('cov_df shape: ', cov_df.shape)
print(cov_df.head())

#count unique number of patient ID
print('Number of unique values in PATNO column: ',cov_df['PATNO'].nunique())

Unique values in White column: 
 ['Yes' 'No' nan 'Unknown or not reported']
Null value in each column: 
 Plate                                  0
PATNO                                 50
SEX                                    0
DIAGNOSIS                              0
age_at_consent                       110
age_at_diagnosis                    3493
Hispanic_or_Latino                   117
AmericanIndian_or_AlaskaNative       117
Asian                                117
Black_or_AfricanAmerican             117
Hawaiian_or_OtherPacificIslander     117
White                                117
Race_not_specified                   117
dtype: int64
Value counts in White column: 
 Yes                        4564
No                          195
Unknown or not reported      15
Name: White, dtype: int64
cov_df shape:  (4564, 5)
    PATNO     SEX DIAGNOSIS  age_at_consent White
0  3174.0    Male        PD       51.162218   Yes
1  3201.0  Female   Control       64.832307   Yes
2  3057.0  Female   

In [9]:
##Check how many patient ID are duplicates and the duplicated records are identical or different
#count how many times patient number is duplicated
size = cov_df.groupby(cov_df['PATNO'].tolist(),as_index=False).size()
print('number of patient ID that has duplicate: ', len(size))

#check how many IDs are only count as 1
size_1 = size.query("size==1")
print('number of patient ID that has no duplicate: ', len(size_1))

#print a couple IDs that have duplicated to see what is going on

size_2 = cov_df.query("PATNO==92834.0")
print(size_2.head())

#print a couple IDs that have duplicated to see what is going on

size_3 = cov_df.query("PATNO==3001.0")
print(size_3.head())

#groupby patient number
df_1 = cov_df.copy()
df_1 = df_1.groupby(['PATNO']).first().reset_index()
df_1

number of patient ID that has duplicate:  1541
number of patient ID that has no duplicate:  449
        PATNO   SEX  DIAGNOSIS  age_at_consent White
947   92834.0  Male  Prodromal       66.748802   Yes
4049  92834.0  Male  Prodromal       66.748802   Yes
4061  92834.0  Male  Prodromal       66.748802   Yes
       PATNO   SEX DIAGNOSIS  age_at_consent White
872   3001.0  Male        PD       65.084189   Yes
902   3001.0  Male        PD       65.084189   Yes
912   3001.0  Male        PD       65.084189   Yes
942   3001.0  Male        PD       65.084189   Yes
2680  3001.0  Male        PD       65.084189   Yes


,PATNO,SEX,DIAGNOSIS,age_at_consent,White
0,3000.0,Female,Control,69.998631,Yes
1,3001.0,Male,PD,65.084189,Yes
2,3002.0,Female,PD,68.161533,Yes
3,3003.0,Female,PD,57.160849,Yes
4,3004.0,Male,Control,60.161533,Yes
...,...,...,...,...,...
1536,85242.0,Male,Prodromal,63.915127,Yes
1537,90456.0,Male,Prodromal,82.496920,Yes
1538,91837.0,Male,Prodromal,74.086242,Yes
1539,92490.0,Male,Prodromal,72.914442,Yes


In [10]:
#drop plate and age_at_diagnosis columns
cov = df_1[['PATNO','SEX', 'DIAGNOSIS','age_at_consent']]
#rename columns
cov.columns = ['ID', 'MALE', 'PHENO','AGE']
#drop rows where ID is NaN
cov = cov[cov['ID'].notna()]

#remove .0 from ID
cov['ID'] = cov['ID'].apply(lambda f: format(f, '.0f'))

#add pp. on PATNO
cov['ID'] = 'PP-' + cov['ID'].astype(str)

#convert Female to 0, Male to 1
cov['MALE'] = cov['MALE'].map({'Female': 0, 'Male': 1})

#remove .0 from MALE
cov['MALE'] = cov['MALE'].apply(lambda f: format(f, '.0f'))

#round AGE values
cov.AGE = cov.AGE.round()
                      
print(cov.head())

#check PHENO choices
print('Unique values in PHENO: ', cov.PHENO.unique())

#count each value in PHENO column
print('Count of each value in PHENO: ', cov.PHENO.value_counts())

        ID MALE    PHENO   AGE
0  PP-3000    0  Control  70.0
1  PP-3001    1       PD  65.0
2  PP-3002    0       PD  68.0
3  PP-3003    0       PD  57.0
4  PP-3004    1  Control  60.0
Unique values in PHENO:  ['Control' 'PD' 'SWEDD' 'Prodromal' 'Genetic Registry' 'Genetic Cohort']
Count of each value in PHENO:  Genetic Registry    424
Genetic Cohort      412
PD                  403
Control             184
SWEDD                60
Prodromal            58
Name: PHENO, dtype: int64


Pheno data

In [11]:
#open pheno file DEMOG_DIAG.csv
pheno = pd.read_csv(f"{datadir}/DEMOG_DIAG.csv", index_col=False )

print('shape of pheno data:', pheno.shape)

#grab PATNO, RECRUIT, FEMALE, YEARSEDUC, FAMILY_HISTORY, DX_INIT
pheno_info = pheno[['PATNO','RECRUIT', 'FEMALE', 'YEARSEDUC', 'FAMILY_HISTORY', 'DX_INIT']]

#check unique values from each column
print('Unique values in RECRUIT:',pheno.RECRUIT.unique())
print('Unique values in YEARSEDUC:',pheno.YEARSEDUC.unique())
print('Unique values in FAMILY_HISTORY:',pheno.FAMILY_HISTORY.unique())
print('Unique values in DX_INIT:',pheno.DX_INIT.unique())

#check nan numbers in each column 
print('null value in each column: \n', pheno_info.isna().sum())
print(pheno_info.head())

#count each value in PHENO column
print('Value counts in DX_INIT:',pheno_info.DX_INIT.value_counts())
print('Number of unique values in PATNO:', pheno_info.PATNO.nunique())

#drop RECRUIT column
pheno_df = pheno_info[['PATNO', 'FEMALE', 'YEARSEDUC', 'FAMILY_HISTORY', 'DX_INIT']]
#rename columns
pheno_df.columns = ['ID', 'FEMALE', 'EDUCATION','FAMILY_HISTORY', 'PHENO']

#add pp. on ID
pheno_df['ID'] = 'PP-' + pheno_df['ID'].astype(str)

#remove .0 from EDUCATION and FAMILY HISTORY
pheno_df['EDUCATION'] = pheno_df['EDUCATION'].apply(lambda f: format(f, '.0f'))
pheno_df['FAMILY_HISTORY'] = pheno_df['FAMILY_HISTORY'].apply(lambda f: format(f, '.0f'))
                 
pheno_df

shape of pheno data: (2154, 14)
Unique values in RECRUIT: ['PD' 'HC' 'SWEDD' 'PRODROMA' 'GENUN' 'REGPD' 'REGUN' 'GENPD']
Unique values in YEARSEDUC: [18. 19. 16. 14. 13. 12. 20. 23. 17. 15. 21. 11. 24. 26. 25.  9. 10. 22.
  8.  5. nan  3.  0.  7.  4.  6.  2. 30. 27.  1. 28.]
Unique values in FAMILY_HISTORY: [ 0.  1. nan]
Unique values in DX_INIT: ['iPD' 'NoDisease' 'ET' 'OtherD' 'Psychogenic' 'DLB' 'PrdrNM' 'PrdrM'
 'Hemi_PKS' 'VaPS']
null value in each column: 
 PATNO              0
RECRUIT            0
FEMALE             0
YEARSEDUC          1
FAMILY_HISTORY    42
DX_INIT            0
dtype: int64
   PATNO RECRUIT  FEMALE  YEARSEDUC  FAMILY_HISTORY    DX_INIT
0   3400      PD       1       18.0             0.0        iPD
1   3401      HC       1       19.0             0.0  NoDisease
2   3402   SWEDD       0       16.0             0.0        iPD
3   3403      PD       0       18.0             0.0        iPD
4   3404      HC       1       14.0             0.0  NoDisease
Value counts in

<ipython-input-11-bdd1b7868e1a>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pheno_df['ID'] = 'PP-' + pheno_df['ID'].astype(str)
<ipython-input-11-bdd1b7868e1a>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pheno_df['EDUCATION'] = pheno_df['EDUCATION'].apply(lambda f: format(f, '.0f'))
<ipython-input-11-bdd1b7868e1a>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,ID,FEMALE,EDUCATION,FAMILY_HISTORY,PHENO
0,PP-3400,1,18,0,iPD
1,PP-3401,1,19,0,NoDisease
2,PP-3402,0,16,0,iPD
3,PP-3403,0,18,0,iPD
4,PP-3404,1,14,0,NoDisease
...,...,...,...,...,...
2149,PP-40750,0,16,1,iPD
2150,PP-42381,1,20,1,NoDisease
2151,PP-41513,0,21,1,NoDisease
2152,PP-40767,0,6,1,iPD


In [12]:
#count PD and Control from Diagnosis
PD_pheno = pheno_df[pheno_df['PHENO'].str.contains('iPD')]
print("Number of PD in pheno data:" , PD_pheno['PHENO'].count())

Control_pheno = pheno_df[pheno_df['PHENO'].str.contains('NoDisease')]
print("Number of Control in pheno data:" , Control_pheno['PHENO'].count())

#only keep the rows contain iPD and NoDisease in DX_INIT
word_list = ["iPD", "NoDisease"]
pheno_df = pheno_df[pheno_df['PHENO'].str.contains('|'.join(word_list))]
pheno_df = pheno_df.reset_index(drop=True)
print("shape of pheno_df:", pheno_df.shape)
print(pheno_df.head())

#count each value in PHENO column
print("Value counts in pheno data:" , pheno_df.PHENO.value_counts())

Number of PD in pheno data: 1040
Number of Control in pheno data: 929
shape of pheno_df: (1969, 5)
        ID  FEMALE EDUCATION FAMILY_HISTORY      PHENO
0  PP-3400       1        18              0        iPD
1  PP-3401       1        19              0  NoDisease
2  PP-3402       0        16              0        iPD
3  PP-3403       0        18              0        iPD
4  PP-3404       1        14              0  NoDisease
Value counts in pheno data: iPD          1040
NoDisease     929
Name: PHENO, dtype: int64


In [13]:
#merge pheno_df with cov
cov = cov.drop(['PHENO'], axis=1)
pheno_df = pheno_df.drop(['FEMALE'], axis=1)
pheno_cov = pheno_df.merge(cov, on='ID', how='inner')

#convert PD to 1, Control to 0
pheno_id = pheno_cov.copy()
pheno_id['PHENO'] = pheno_id['PHENO'].map({'NoDisease': 0, 'iPD': 1})

#leave only PHENO=1 to only include only cases, PHENO=0 for controls
cov_case = pheno_id.loc[pheno_id['PHENO'] == 1]
cov_cont = pheno_id.loc[pheno_id['PHENO'] == 0]

#drop Pheno column
cov_case = cov_case.drop(['PHENO'], axis=1)
cov_cont = cov_cont.drop(['PHENO'], axis=1)

#reset index
cov_case = cov_case.reset_index(drop=True)
cov_cont = cov_cont.reset_index(drop=True)

print("shape of cov_case:", cov_case.shape)
print(cov_case.head())

print("shape of cov_cont:", cov_cont.shape)
print(cov_cont.head())

#count each value in PHENO column
print("Value counts in pheno_id data:" , pheno_id.PHENO.value_counts())

shape of cov_case: (834, 5)
        ID EDUCATION FAMILY_HISTORY MALE   AGE
0  PP-3400        18              0    0  39.0
1  PP-3402        16              0    1  46.0
2  PP-3403        18              0    1  69.0
3  PP-3406        18              0    1  35.0
4  PP-3407        16              0    1  66.0
shape of cov_cont: (626, 5)
        ID EDUCATION FAMILY_HISTORY MALE   AGE
0  PP-3401        19              0    0  56.0
1  PP-3404        14              0    0  56.0
2  PP-3405        18              0    0  63.0
3  PP-3100        16              0    0  69.0
4  PP-3151        13              0    1  58.0
Value counts in pheno_id data: 1    834
0    626
Name: PHENO, dtype: int64


Check plink files and make 10 genomic PCs

In [20]:
#check number of IDs in original .fam file
!cat {datadir}/hits_plus_tags_AMPPD_Euros.fam | wc -l
#check .fam file
!head /{datadir}/hits_plus_tags_AMPPD_Euros.fam

#check number of IDs in original .bim file
!cat {datadir}/hits_plus_tags_AMPPD_Euros.bim | wc -l
#check .bim file
!head /{datadir}/hits_plus_tags_AMPPD_Euros.bim

2927
BF-1001 BF-1001 0 0 0 -9
BF-1002 BF-1002 0 0 0 -9
BF-1003 BF-1003 0 0 0 -9
BF-1004 BF-1004 0 0 0 -9
BF-1005 BF-1005 0 0 0 -9
BF-1006 BF-1006 0 0 0 -9
BF-1008 BF-1008 0 0 0 -9
BF-1009 BF-1009 0 0 0 -9
BF-1010 BF-1010 0 0 0 -9
BF-1011 BF-1011 0 0 0 -9
305138
1	rs200683566	0	13838	T	C
1	rs370886505	0	14397	C	CTGT
1	rs375086259	0	14653	T	C
1	rs79585140	0	14907	G	A
1	rs199856693	0	14933	A	G
1	rs71252250	0	15118	G	A
1	rs201635489	0	15447	G	A
1	rs201026389	0	16125	G	T
1	rs78588380	0	16257	C	G
1	rs200736374	0	16288	G	C


In [ ]:
#extract genetic PCs from expression data 
!plink --bfile /{datadir}/hits_plus_tags_AMPPD_Euros --pca 10 --out /{datadir}/PCA_1

In [14]:
#read genetic PCs
pcs_df = pd.read_csv(f"{datadir}/PCA_1.eigenvec", sep=' ', usecols=[1,2,3,4,5,6,7,8,9,10,11], names=['ID','PC1_gen','PC2_gen','PC3_gen','PC4_gen','PC5_gen','PC6_gen','PC7_gen','PC8_gen','PC9_gen','PC10_gen'])
pcs_df

,ID,PC1_gen,PC2_gen,PC3_gen,PC4_gen,PC5_gen,PC6_gen,PC7_gen,PC8_gen,PC9_gen,PC10_gen
0,BF-1001,-0.013383,0.026178,-0.012396,0.004339,-0.005004,0.007554,-0.002626,0.022212,0.011273,-0.004516
1,BF-1002,-0.011969,0.014639,-0.026859,-0.000687,-0.003197,-0.005286,-0.000583,0.015597,0.002706,-0.007646
2,BF-1003,-0.010549,0.021005,0.014955,-0.000793,-0.001582,0.008044,0.002457,-0.000282,-0.002610,0.001135
3,BF-1004,-0.012569,0.023211,-0.006541,0.003494,-0.011989,-0.000027,-0.004490,0.037969,0.016873,-0.008314
4,BF-1005,-0.010763,0.011671,-0.020893,-0.012810,-0.007227,-0.003492,-0.005223,-0.016002,-0.023424,0.000279
...,...,...,...,...,...,...,...,...,...,...,...
2922,PP-85242,-0.009681,-0.012745,0.022996,-0.000655,-0.001486,-0.002943,0.009491,-0.003265,0.019764,0.008159
2923,PP-90456,0.032165,0.001445,0.039731,-0.004689,-0.011589,0.001291,-0.004769,-0.004276,-0.007312,0.010096
2924,PP-91837,-0.012103,-0.001567,-0.020199,0.001134,-0.008514,-0.004052,-0.009519,0.014668,-0.027068,-0.003649
2925,PP-92490,-0.017216,0.001212,0.002467,-0.016545,-0.027899,-0.001444,-0.004236,-0.005391,-0.020077,0.000391


In [18]:
#merge pcs_df into covariate_df
covar_df_case = cov_case.merge(pcs_df, on='ID', how='left')
covar_df_cont = cov_cont.merge(pcs_df, on='ID', how='left')

#convert nan in FAMILY_HISTORY to null
covar_df_case['FAMILY_HISTORY'] = covar_df_case['FAMILY_HISTORY'].replace('nan',np.nan)
covar_df_cont['FAMILY_HISTORY'] = covar_df_cont['FAMILY_HISTORY'].replace('nan',np.nan)

#check nan numbers in each column 
print('null value in each column in case: \n', covar_df_case.isna().sum())
print('null value in each column in control: \n', covar_df_cont.isna().sum())

null value in each column in case: 
 ID                  0
EDUCATION           0
FAMILY_HISTORY     13
MALE                0
AGE                 0
PC1_gen           101
PC2_gen           101
PC3_gen           101
PC4_gen           101
PC5_gen           101
PC6_gen           101
PC7_gen           101
PC8_gen           101
PC9_gen           101
PC10_gen          101
dtype: int64
null value in each column in control: 
 ID                 0
EDUCATION          0
FAMILY_HISTORY     7
MALE               0
AGE                0
PC1_gen           91
PC2_gen           91
PC3_gen           91
PC4_gen           91
PC5_gen           91
PC6_gen           91
PC7_gen           91
PC8_gen           91
PC9_gen           91
PC10_gen          91
dtype: int64


In [19]:
#drop rows that has null values
covar_df_case = covar_df_case[covar_df_case['PC1_gen'].notna()]
covar_df_cont = covar_df_cont[covar_df_cont['PC1_gen'].notna()]

#reset index
covar_df_case = covar_df_case.reset_index(drop=True)
covar_df_cont = covar_df_cont.reset_index(drop=True)

#rename ID to FID
covar_df_case = covar_df_case.rename(columns={'ID':'FID'})
covar_df_cont = covar_df_cont.rename(columns={'ID':'FID'})

#grab ID column and duplicate into two columns to compare with .fam file FID IID
covar_df_case.insert(1, 'IID', covar_df_case['FID'])
covar_df_cont.insert(1, 'IID', covar_df_cont['FID'])

#replace NaN withmean for nonbinary and median for binary column
covar_df_case['FAMILY_HISTORY'] = covar_df_case['FAMILY_HISTORY'].fillna(covar_df_case['FAMILY_HISTORY'].median())
covar_df_cont['FAMILY_HISTORY'] = covar_df_cont['FAMILY_HISTORY'].fillna(covar_df_cont['FAMILY_HISTORY'].median())

#make ID as list to trim expression data later
covar_df_case_list = covar_df_case['FID']
covar_df_cont_list = covar_df_cont['FID']

print('shape of covar_df_case', covar_df_case.shape)
print(covar_df_case.head())
print('shape of covar_df_cont', covar_df_cont.shape)
print(covar_df_cont.head())

shape of covar_df_case (733, 16)
       FID      IID EDUCATION FAMILY_HISTORY MALE   AGE   PC1_gen   PC2_gen  \
0  PP-3400  PP-3400        18              0    0  39.0 -0.004662 -0.023299   
1  PP-3402  PP-3402        16              0    1  46.0  0.011903 -0.023524   
2  PP-3403  PP-3403        18              0    1  69.0 -0.014213 -0.025636   
3  PP-3406  PP-3406        18              0    1  35.0 -0.008313 -0.010611   
4  PP-3407  PP-3407        16              0    1  66.0  0.033813 -0.012696   

    PC3_gen   PC4_gen   PC5_gen   PC6_gen   PC7_gen   PC8_gen   PC9_gen  \
0  0.006330 -0.001153  0.003980 -0.002586 -0.000425 -0.007179 -0.020155   
1 -0.001647  0.005045 -0.003863 -0.003192 -0.003624  0.000327  0.012216   
2 -0.002261 -0.003854 -0.017373  0.002293 -0.002750  0.023918  0.020800   
3  0.017661 -0.010679 -0.007702 -0.004243 -0.004153  0.002486  0.002382   
4  0.020515 -0.000225 -0.008233 -0.003591  0.002511 -0.004819  0.002946   

   PC10_gen  
0 -0.000930  
1 -0.006821  

In [21]:
#check unique values and data type in each column
for i in covar_df_case.columns[2:6]:
    print('Unique value of ' + i + ':', covar_df_case[i].unique())
    print('Data types of '  + i + ':',covar_df_case[i].dtypes)

for i in covar_df_cont.columns[2:6]:
    print('Unique value of ' + i + ':', covar_df_cont[i].unique())
    print('Data types of '  + i + ':',covar_df_cont[i].dtypes)
    
#change float to int in column EDUCATION, FAMILY_HISTORY, and UPSIT

##col=(covariate.dtypes=='float64')
col_cont=covar_df_case.columns[2:6]
covar_df_case.loc[:,col_cont]=covar_df_case.loc[:,col_cont].astype(np.int64)
covar_df_cont.loc[:,col_cont]=covar_df_cont.loc[:,col_cont].astype(np.int64)

#normalize covariate values for AGE, EDUCATION columns with min_max_scaling

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
covar_df_case_scale = covar_df_case.copy()
covar_df_case_scale[['AGE', 'EDUCATION']] = scaler.fit_transform(covar_df_case_scale[['AGE', 'EDUCATION']])

covar_df_cont_scale = covar_df_cont.copy()
covar_df_cont_scale[['AGE', 'EDUCATION']] = scaler.fit_transform(covar_df_cont_scale[['AGE', 'EDUCATION']])

# apply the min-max scaling in Pandas using the .min() and .max() methods
#def min_max_scaling(df):
    # copy the dataframe
 #   df_norm = df.copy()
    # apply min-max scaling
  #  for column in df_norm.iloc[:, [2, 5]]:
   #     df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min()) 
        
    #return df_norm
    
# call the min_max_scaling function
#covar_df_case_scale = min_max_scaling(covar_df_case)
print('shape of covariate data for case:', covar_df_case_scale.shape)
print(covar_df_case_scale.head())

#covar_df_cont_scale = min_max_scaling(covar_df_cont)
print('shape of covariate data for control:', covar_df_cont_scale.shape)
print(covar_df_cont_scale.head())

Unique value of EDUCATION: [18 16 13 14 12 23 17 21 24 19 26 20 15 10  8 11 22  9  5 25  3  7  6  0
  2  1]
Data types of EDUCATION: int64
Unique value of FAMILY_HISTORY: [0 1]
Data types of FAMILY_HISTORY: int64
Unique value of MALE: [0 1]
Data types of MALE: int64
Unique value of AGE: [39 46 69 35 66 58 52 64 50 72 70 68 55 74 67 75 60 73 57 48 62 56 80 83
 63 59 65 54 41 76 51 45 40 71 47 78 38 61 53 49 77 85 79 34 43 42 44 37
 82 88 81 94 87 33 92 84 90]
Data types of AGE: int64
Unique value of EDUCATION: [19 14 18 16 13 20 12 17 15  9 22 21 11 23  8 24  7  2 30 25 10 26 27  6]
Data types of EDUCATION: int64
Unique value of FAMILY_HISTORY: [0 1]
Data types of FAMILY_HISTORY: int64
Unique value of MALE: [0 1]
Data types of MALE: int64
Unique value of AGE: [56 63 69 58 52 75 65 32 41 60 70 72 76 53 38 80 79 77 45 68 54 49 50 61
 71 82 42 47 31 59 64 62 74 46 66 73 55 43 44 48 33 57 78 67 83 81 51 36
 27 35 29 30 25 37 39 34 87 40 84 22 19 26]
Data types of AGE: int64
shape of covaria

In [ ]:
#save covariate as txt --> this is used to trim expression data
covar_df_case_list.to_csv(r'./data_folder/patient_id_case.txt', sep='\t' ,index=False)
covar_df_cont_list.to_csv(r'./data_folder/patient_id_control.txt', sep='\t' ,index=False)

Coordinate data

In [2]:
#open te location file hg38_rmsk_TE.gtf.locInd.locations
chr_pos_df = pd.read_csv(f"{datadir}/hg38_gene_annotation.gtf.locations", sep='\t', header=None, names=('X.Chr','start','end','ID'))

print('shape of gene location data:', chr_pos_df.shape)

#remove gene_id, duplicated ENST number, and "",;
chr_pos_df['ID'] = chr_pos_df.ID.str.extract(r'gene_id .*?"([^"]+)";.+')

#check nan numbers in each column 
print('null value in each column: \n', chr_pos_df.isna().sum())

#checke chromosome inputs in chr_pos_df
unique_chromosom = chr_pos_df['X.Chr'].unique().tolist()
print('unique chromosom:', unique_chromosom)

shape of gene location data: (2535323, 4)
null value in each column: 
 X.Chr    0
start    0
end      0
ID       0
dtype: int64
unique chromosom: ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrM', 'chrX', 'chrY', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrUn_GL000195v1', 'chrUn_GL000213v1', 'chrUn_GL000216v2', 'chrUn_GL000218v1', 'chrUn_GL000219v1', 'chrUn_GL000220v1', 'chrUn_KI270442v1', 'chrUn_KI270744v1', 'chrUn_KI270750v1', 'chr1_GL383518v1_alt', 'chr1_GL383519v1_alt', 'chr1_GL383520v2_alt', 'chr1_KI270759v1_alt', 'chr1_KI270760v1_alt', 'chr1_KI270761v1_alt', 'chr1_KI270762v1_alt', 'chr1_KI270763v1_alt', 'chr1_KI270765v1_alt', 'chr1_KI270766v1_alt', 'chr1_KI270892v1_alt', 'chr1_KN196472v1_fix', 'chr1_KN196473v1_fix', 'chr1_KN196474v1_fix', 'chr1_KN538360v1_fix', 'chr1_KN538361v1_fix', 'chr1_KQ031383v1_fix', 'chr1_KQ458382v1_alt', 'chr1_KQ458383v1_alt', 'chr1_KQ458384v1_alt', 

In [3]:
#count how many M are in coord
m_coords = chr_pos_df[chr_pos_df['X.Chr'].str.contains('M')]
print("Number of M chromosom in coordinate data:" , m_coords['X.Chr'].count())

#count how many y are in coord
y_coords = chr_pos_df[chr_pos_df['X.Chr'].str.contains('Y')]
print("Number of Y chromosom in coordinate data:" , y_coords['X.Chr'].count())

#count how many X are in coord
x_coords = chr_pos_df[chr_pos_df['X.Chr'].str.contains('X')]
print("Number of X chromosom in coordinate data:" , x_coords['X.Chr'].count())

#count how many Un are in coord
un_coords = chr_pos_df[chr_pos_df['X.Chr'].str.contains('Un')]
print("Number of Unknown chromosom in coordinate data:" , un_coords['X.Chr'].count())

#drop all the rows contain M, X, Y in X.Chr 
word_list = ["M", "X", "Y", "Un"]
coord_df = chr_pos_df[~chr_pos_df['X.Chr'].str.contains('|'.join(word_list))]
coord_df = coord_df.reset_index(drop=True)
print("shape of coord_df:", coord_df.shape)
print(coord_df.head())

#check how many ID are unique
print("Number of unique ID in coord_df:", coord_df['ID'].nunique())

#count how many times patient number is duplicated
duplicate = coord_df.groupby(coord_df['ID'].tolist(),as_index=False).size()
print("Number count of ID in coord_df:", duplicate)

#check how many IDs are only count as 1
duplicate_1 = duplicate.query("size==1")
print('number of transcript ID that has no duplicate: ', len(duplicate_1))

Number of M chromosom in coordinate data: 76
Number of Y chromosom in coordinate data: 7634
Number of X chromosom in coordinate data: 74651
Number of Unknown chromosom in coordinate data: 82
shape of coord_df: (2452880, 4)
  X.Chr  start    end                 ID
0  chr1  11869  12227  ENST00000456328.2
1  chr1  12613  12721  ENST00000456328.2
2  chr1  13221  14409  ENST00000456328.2
3  chr1  12010  12057  ENST00000450305.2
4  chr1  12179  12227  ENST00000450305.2
Number of unique ID in coord_df: 239497
Number count of ID in coord_df:                      index  size
0       ENST00000000233.10    14
1        ENST00000000412.8    15
2       ENST00000000442.11    15
3        ENST00000001008.6    22
4        ENST00000001146.6    14
...                    ...   ...
239492   ENST00000673604.1    20
239493   ENST00000673615.1    37
239494   ENST00000673616.1    16
239495   ENST00000673620.1     9
239496   ENST00000673621.1    35

[239497 rows x 2 columns]
number of transcript ID that has no 

In [5]:
#print a couple IDs that have duplicated to see what is going on
exp_coord_1 = coord_df[coord_df['ID'].str.contains('ENST00000673620.1')]
print('number of ENST00000673620.1 in coord data: ', exp_coord_1.shape[0])

#print a couple IDs that have duplicated to see what is going on
exp_coord_2 = coord_df[coord_df['ID'].str.contains('ENST00000000233.10')]
print('number of ENST00000000233.10 in coord data: ', exp_coord_2.shape[0])

#count unique values in X.Chr in same ID
grouped_df = coord_df.groupby('ID')
grouped_df = grouped_df.agg({"X.Chr": "nunique"})
grouped_df = grouped_df.reset_index()
print(grouped_df)
#check how many have 1 kind of X.Chr
print('number of chromosomes that only have 1 kind in each ID: ', len(grouped_df[grouped_df['X.Chr'] == 1]))
##safe to assume duplicated IDs are in same Chromosome

#merge duplicated ID into 1 and take smallest start and biggest end
coord_comb = coord_df.groupby(['ID']).agg({'start': 'min', 'end': 'max', 'X.Chr': 'first'}).reset_index()

#function to move columns
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

#rearrange Gene_Symbol, Chr, Start, Stop columns right nextg to index column 
coord_comb = movecol(coord_comb, cols_to_move=['X.Chr'], ref_col='ID', place='Before')
coord_comb = movecol(coord_comb, cols_to_move=['ID'], ref_col='end', place='After')
coord_comb

number of ENST00000673620.1 in coord data:  9
number of ENST00000000233.10 in coord data:  14
                        ID  X.Chr
0       ENST00000000233.10      1
1        ENST00000000412.8      1
2       ENST00000000442.11      1
3        ENST00000001008.6      1
4        ENST00000001146.6      1
...                    ...    ...
239492   ENST00000673604.1      1
239493   ENST00000673615.1      1
239494   ENST00000673616.1      1
239495   ENST00000673620.1      1
239496   ENST00000673621.1      1

[239497 rows x 2 columns]
number of chromosomes that only have 1 kind in each ID:  239497


,X.Chr,start,end,ID
0,chr7,127588411,127591700,ENST00000000233.10
1,chr12,8940361,8949645,ENST00000000412.8
2,chr11,64305524,64316743,ENST00000000442.11
3,chr12,2794970,2805423,ENST00000001008.6
4,chr2,72129238,72148038,ENST00000001146.6
...,...,...,...,...
239492,chr11,47242846,47248840,ENST00000673604.1
239493,chr8,132063785,132111159,ENST00000673615.1
239494,chr19_KV575251v1_alt,105193,116143,ENST00000673616.1
239495,chr3,15601726,15672955,ENST00000673620.1


In [6]:
#check if any ID have end-start > 1000000

coord_comb_1 = coord_comb[((coord_comb['end'] - coord_comb['start']) > 1000000)]
print('shape of coordinate data greater than 1 mega:', coord_comb_1.shape)
print(coord_comb_1)

coord_comb_2 = coord_comb[((coord_comb['end'] - coord_comb['start']) > 2000000)]
print('shape of coordinate data greater than 2 mega:', coord_comb_2.shape)
print(coord_comb_2)

shape of coordinate data greater than 1 mega: (170, 4)
        X.Chr      start        end                  ID
630     chr20   13995369   16053197   ENST00000217246.8
4519    chr16   82626965   83795376  ENST00000268613.14
5436     chr4   92303966   93774566   ENST00000282020.9
6839     chr8  112222928  113436939  ENST00000297405.10
11717    chr2  213284464  214410501  ENST00000331683.10
...       ...        ...        ...                 ...
218934   chr8   31639304   32728077   ENST00000651149.1
219107   chr8   31640659   32855666   ENST00000651335.1
220377   chr8   31639245   32774046   ENST00000652698.1
223686   chr6   97305577   98398863   ENST00000656098.1
236274   chr6   97305553   98398783   ENST00000669069.1

[170 rows x 4 columns]
shape of coordinate data greater than 2 mega: (12, 4)
        X.Chr      start        end                 ID
630     chr20   13995369   16053197  ENST00000217246.8
17695    chr7  146116801  148420998  ENST00000361727.8
24177   chr11   83457919   856

In [7]:
#seperate any character after chr number in X.Chr and put it in a new column
coord_comb['Chr_info'] = coord_comb['X.Chr']
coord_comb['X.Chr'] = coord_comb['X.Chr'].str.split('_').str[0]

#seperate information in Chr_info into info_1 and info_2
coord_comb['info_2'] = coord_comb['Chr_info']
coord_comb['info_2'] = coord_comb['Chr_info'].str.split('_').str[2]
coord_comb['Chr_info'] = coord_comb['Chr_info'].str.split('_').str[1]
coord_comb = coord_comb.rename(columns={'Chr_info': 'info_1'})

#delete chr from X.Chr
coord_comb['X.Chr'] = coord_comb['X.Chr'].str.replace('chr', '')

#convert X.Chr to int
coord_comb['X.Chr'] = coord_comb['X.Chr'].astype(np.int64)

#arrange by assending order of X.Chr
coord_comb = coord_comb.sort_values(by = ['X.Chr'], ascending=True).reset_index(drop=True)

#delete info_1, info_2
coord_comb_3 = coord_comb.drop(['info_1', 'info_2'], axis=1)
coord_comb_3

,X.Chr,start,end,ID
0,1,90916390,91022014,ENST00000467231.5
1,1,15456732,15472091,ENST00000359621.5
2,1,21233570,21238350,ENST00000470394.1
3,1,54032019,54036802,ENST00000470395.1
4,1,151266308,151266725,ENST00000470396.1
...,...,...,...,...
239492,22,30555938,30560759,ENST00000447224.5
239493,22,44196098,44219533,ENST00000472551.1
239494,22,24632915,24647760,ENST00000447261.6
239495,22,20967248,20974096,ENST00000426113.5


In [8]:
#save coord_df as txt -- run only once for download
coord_comb.to_csv(r'./data_folder/transcript_coordinate_long.txt', sep='\t' ,index=False)

#save coord_df as txt -- run only once for download
coord_comb_3.to_csv(r'./data_folder/transcript_coordinate_short.txt', sep='\t' ,index=False)

In [9]:
#get ID as a list and save to compare with dataset
gene_list = coord_comb_3['ID']
print("number of genes in gene_list:", len(gene_list))

#create gene_list as list to drop columns in exp_case 
gene_list_1 = coord_comb_3['ID'].tolist()

number of genes in gene_list: 239497


In [ ]:
#save gene_list as txt -- run only once for download
gene_list.to_csv(r'./data_folder/transcript_id_list.txt', sep='\t' ,index=False)

Expression data (baseline & transcript ID only)

In [ ]:
#split data into 4 in equal size to open easily
!split -l $(( $( wc -l < ./data_folder/baseline_GENE_TE_norm_adjusted.cntTable ) / 4 + 1 )) ./data_folder/baseline_GENE_TE_norm_adjusted.cntTable ./data_folder/baseline_GENE_TE_norm_adjusted.cntTable

In [3]:
#check the number of rows 
!cat ./data_folder/baseline_GENE_TE_norm_adjusted.cntTableaa | wc -l
!cat ./data_folder/baseline_GENE_TE_norm_adjusted.cntTableab | wc -l
!cat ./data_folder/baseline_GENE_TE_norm_adjusted.cntTableac | wc -l
!cat ./data_folder/baseline_GENE_TE_norm_adjusted.cntTablead | wc -l

1042229
1042229
1042229
1042225


In [ ]:
#read column names (header) (patient number, which is our sample ID) and output as text file
!head -1 ./data_folder/baseline_GENE_TE_norm_adjusted.cntTable > ./data_folder/baseline_id.txt

In [4]:
#read baseline dataset header
base = pd.read_csv(f"{datadir}/baseline_id.txt", engine='c', sep= '\t')
base_header = base.columns.tolist()
len(base_header)

1565

In [11]:
#read index and output as text file
!awk '{print $1}' ./data_folder/baseline_GENE_TE_norm_adjusted.cntTableaa > ./data_folder/aa_trans.txt
!awk '{print $1}' ./data_folder/baseline_GENE_TE_norm_adjusted.cntTableab > ./data_folder/ab_trans.txt
!awk '{print $1}' ./data_folder/baseline_GENE_TE_norm_adjusted.cntTableac > ./data_folder/ac_trans.txt
!awk '{print $1}' ./data_folder/baseline_GENE_TE_norm_adjusted.cntTablead > ./data_folder/ad_trans.txt

In [20]:
#read baseline_2 data
aa_trans = pd.read_csv(f"{datadir}/aa_trans.txt", header=None, names=['ID'], engine='c',sep='\t')
ab_trans = pd.read_csv(f"{datadir}/ab_trans.txt", header=None, names=['ID'], engine='c',sep='\t')
ac_trans = pd.read_csv(f"{datadir}/ac_trans.txt", header=None, names=['ID'], engine='c',sep='\t')
ad_trans = pd.read_csv(f"{datadir}/ad_trans.txt", header=None, names=['ID'], engine='c',sep='\t')

#print a couple IDs that have duplicated to see what is going on
aa_trans_enst = aa_trans[aa_trans['ID'].str.contains('ENST')]
print('number of ENST in Tableaa expression data: ', aa_trans_enst.shape[0])

#print a couple IDs that have duplicated to see what is going on
ab_trans_enst = ab_trans[ab_trans['ID'].str.contains('ENST')]
print('number of ENST in Tableab expression data: ', ab_trans_enst.shape[0])

#print a couple IDs that have duplicated to see what is going on
ac_trans_enst = ac_trans[ac_trans['ID'].str.contains('ENST')]
print('number of ENST in Tableac expression data: ', ac_trans_enst.shape[0])

#print a couple IDs that have duplicated to see what is going on
ad_trans_enst = ad_trans[ad_trans['ID'].str.contains('ENST')]
print('number of ENST in Tablead expression data: ', ad_trans_enst.shape[0])

##will process only Tableaa since all ENST values are in there

number of ENST in Tableaa expression data:  227021
number of ENST in Tableab expression data:  0
number of ENST in Tableac expression data:  0
number of ENST in Tablead expression data:  0


In [26]:
#read patient_id
case_id = pd.read_csv(f"{datadir}/patient_id_case.txt",  engine='c',sep='\t')
case_id_list = case_id['FID'].tolist()
print('Number of case ID: ', len(case_id))

#read patient_id
cont_id = pd.read_csv(f"{datadir}/patient_id_control.txt",  engine='c',sep='\t')
cont_id_list = cont_id['FID'].tolist()
print('Number of control ID: ', len(cont_id))

#read transcript_id_list
aa_trans_enst_list = aa_trans_enst['ID'].tolist()
print('Number of transcript ID: ', len(aa_trans_enst_list))

#read transcript_id_list 
tr = pd.read_csv(f"{datadir}/transcript_id_list.txt", sep='\t')
print('Number of transcript ID that has coordinate data: ', len(tr))

Number of case ID:  733
Number of control ID:  535
Number of transcript ID:  227021
Number of transcript ID that has coordinate data:  239497


In [27]:
#read baseline_1 data
base_aa = pd.read_csv(f"{datadir}/baseline_GENE_TE_norm_adjusted.cntTableaa", engine='c',sep='\t')

#round the decimal numbers to 6 digits
base_aa = base_aa.round(decimals=6)

#change ID format as PP-
base_aa.columns = base_aa.columns.str.replace('pp.', 'PP-')

base_aa.head()

<ipython-input-27-f656285d6144>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  base_aa.columns = base_aa.columns.str.replace('pp.', 'PP-')


,PP-10874,PP-12224,PP-12499,PP-12593,PP-13039,PP-13424,PP-14281,PP-14331,PP-14426,PP-15761,...,PP-65008,PP-70239,PP-85062,PP-85236,PP-85242,PP-90456,PP-91097,PP-91837,PP-92490,PP-92834
ENST00000000233.10,1.055457,-0.090322,0.489696,0.108862,1.284662,0.916743,0.528411,0.982363,1.079028,0.541646,...,0.493132,0.888485,0.828482,1.165689,1.536568,0.840764,1.066142,-0.793131,1.626177,1.623303
ENST00000000412.8,3.701636,2.136299,2.517864,2.635514,3.526311,2.515658,2.373977,3.547481,3.384911,2.996052,...,1.673927,2.566919,3.439492,3.236078,2.080992,2.773280,2.492843,0.083720,3.180079,3.450967
ENST00000000442.11,3.143852,1.563251,1.977397,1.849058,2.940845,2.039643,2.220080,2.683409,2.783043,2.126609,...,1.379228,2.192926,2.389890,2.277158,2.153871,2.351454,2.361598,0.228564,2.785307,3.587059
ENST00000001008.6,1.884540,1.494641,1.580101,1.475380,1.637678,1.773128,1.673841,2.092372,2.035153,1.568978,...,1.070522,1.783352,1.980777,1.742746,2.263322,1.626536,1.694174,0.436736,1.396854,2.136774
ENST00000001146.6,-1.545447,-0.430481,-6.233680,-1.583514,-1.749059,-4.335397,-1.708629,-1.778449,-6.682523,-0.996774,...,-1.529708,-4.749588,-3.660453,-7.086976,-7.096427,-1.501998,-2.698144,-0.199452,-6.663842,-3.825158


In [28]:
#drop columns if not case_id_list
base_aa_case = base_aa.drop(columns=[col for col in base_aa[base_aa.columns] if col not in case_id_list])

#reset index
base_aa_case = base_aa_case.reset_index()

#rename index column as ID
base_aa_case = base_aa_case.rename(columns={'index': 'ID'})

#split ID 
base_aa_case['ID'] = base_aa_case['ID'].str.split(':').str[0]

#merge tr_list with base_aa_case
base_tr_case = pd.merge(base_aa_case, tr, on=['ID'], how='inner')

#set index
base_tr_case.set_index('ID', inplace=True)

#transpose the dataframe
base_tr_case = base_tr_case.T

#add ID as index name and reset the index
base_tr_case = base_tr_case.rename_axis('FID').reset_index()

#grab ID column and duplicate into two columns to compare with .fam file FID IID
base_tr_case.insert(1, 'IID', base_tr_case['FID'])

#remove index name
base_tr_case = base_tr_case.rename_axis(None, axis=1)

base_tr_case

,FID,IID,ENST00000000233.10,ENST00000000412.8,ENST00000000442.11,ENST00000001008.6,ENST00000001146.6,ENST00000002125.9,ENST00000002165.11,ENST00000002501.10,...,ENST00000673594.1,ENST00000673597.1,ENST00000673598.1,ENST00000673599.1,ENST00000673600.1,ENST00000673603.1,ENST00000673604.1,ENST00000673615.1,ENST00000673620.1,ENST00000673621.1
0,PP-3001,PP-3001,1.231180,3.278386,2.645443,2.145071,-6.633006,0.959451,2.498851,-4.311078,...,-2.385079,-6.633006,-2.726115,-0.800116,3.760384,-3.463081,-2.932566,-6.633006,-0.190063,0.135178
1,PP-3002,PP-3002,0.112015,2.722850,1.680395,1.477788,-0.373412,0.282856,1.920547,-1.912931,...,-2.695340,-3.879764,-3.638756,-1.557836,3.278272,-3.251733,-3.879764,-1.981644,-0.544780,-0.305773
2,PP-3003,PP-3003,0.996263,3.426735,2.620935,1.814124,-3.051490,0.715676,2.425490,-3.582005,...,-2.228368,-6.751930,-3.051490,-0.869287,3.744924,-2.228368,-3.582005,-5.166967,-0.122573,-0.122573
3,PP-3006,PP-3006,0.570453,3.279767,2.629038,1.801636,-1.932047,0.744658,2.283302,-3.806516,...,-2.452880,-6.976441,-2.888979,-0.999162,3.769233,-3.806516,-3.276002,-6.976441,-0.406586,-0.318230
4,PP-3007,PP-3007,0.560225,3.055598,2.140143,1.910574,-5.432242,0.744347,2.293409,-3.557772,...,-2.373348,-7.017204,-2.929741,-0.908680,3.428845,-2.373348,-3.110313,-5.432242,-0.275737,0.201964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,PP-58510,PP-58510,1.036279,3.110402,2.481665,1.684186,-7.748356,0.715169,2.050926,-3.356038,...,-2.020435,-7.748356,-2.890375,-0.805841,3.717721,-4.047916,-3.104500,-7.748356,0.152511,0.290563
701,PP-58858,PP-58858,0.842236,3.181759,2.535167,1.638167,-3.047060,0.877213,2.379205,-3.336567,...,-2.114174,-6.506492,-3.047060,-0.891782,4.249230,-4.921529,-3.047060,-6.506492,-0.030758,0.313687
702,PP-58951,PP-58951,0.904115,2.548815,2.325297,1.230259,-6.775365,0.143498,1.551064,-3.315934,...,-2.383048,-6.775365,-3.315934,-0.942475,3.518106,-3.605440,-3.968010,-5.190403,-0.007181,0.537518
703,PP-59398,PP-59398,0.956323,2.863001,2.282723,1.468014,-6.275698,0.363341,1.755888,-4.401229,...,-1.978017,-7.860660,-3.105773,-0.965843,3.265399,-2.368807,-3.216804,-7.860660,0.232097,0.569792


In [31]:
#drop columns if not case_id_list
base_aa_cont = base_aa.drop(columns=[col for col in base_aa[base_aa.columns] if col not in cont_id_list])

#reset index
base_aa_cont = base_aa_cont.reset_index()

#rename index column as ID
base_aa_cont = base_aa_cont.rename(columns={'index': 'ID'})

#split ID 
base_aa_cont['ID'] = base_aa_cont['ID'].str.split(':').str[0]

#merge tr_list with base_aa_case
base_tr_cont = pd.merge(base_aa_cont, tr, on=['ID'], how='inner')

#set index
base_tr_cont.set_index('ID', inplace=True)

#transpose the dataframe
base_tr_cont = base_tr_cont.T

#add ID as index name and reset the index
base_tr_cont = base_tr_cont.rename_axis('FID').reset_index()

#grab ID column and duplicate into two columns to compare with .fam file FID IID
base_tr_cont.insert(1, 'IID', base_tr_cont['FID'])

#remove index name
base_tr_cont = base_tr_cont.rename_axis(None, axis=1)

base_tr_cont

,FID,IID,ENST00000000233.10,ENST00000000412.8,ENST00000000442.11,ENST00000001008.6,ENST00000001146.6,ENST00000002125.9,ENST00000002165.11,ENST00000002501.10,...,ENST00000673594.1,ENST00000673597.1,ENST00000673598.1,ENST00000673599.1,ENST00000673600.1,ENST00000673603.1,ENST00000673604.1,ENST00000673615.1,ENST00000673620.1,ENST00000673621.1
0,PP-14426,PP-14426,1.079028,3.384911,2.783043,2.035153,-6.682523,1.206220,2.915529,-2.775633,...,-1.927636,-6.682523,-2.775633,-0.453705,4.289739,-3.875168,-3.223092,-6.682523,-0.143364,0.137656
1,PP-15761,PP-15761,0.541646,2.996052,2.126609,1.568978,-0.996774,0.338829,2.110755,-2.330198,...,-2.450492,-4.652126,-3.273614,-1.419465,3.289215,-2.450492,-3.514622,-2.886591,-0.344697,-0.564663
2,PP-17608,PP-17608,0.386916,3.019584,2.272599,2.000488,-2.722383,0.572578,1.976001,-4.669915,...,-2.432876,-7.477270,-2.953708,-1.137420,3.265039,-4.017838,-3.389807,-7.477270,0.517083,0.494273
3,PP-3000,PP-3000,0.453616,2.575986,2.197861,1.569525,-2.597812,0.040448,1.679213,-2.797121,...,-2.342555,-5.230080,-3.464546,-1.248228,2.954795,-3.464546,-3.645118,-4.382084,0.010234,-0.005114
4,PP-3004,PP-3004,0.554084,2.515141,1.921878,1.236488,-1.187848,0.534116,1.882937,-3.382864,...,-2.421338,-4.823436,-3.238474,-1.748148,3.676979,-2.986935,-3.930351,-2.273239,-0.247087,-0.008739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,PP-65000,PP-65000,1.270003,2.563567,2.436615,1.568766,-5.550176,-0.289648,1.519498,-3.965213,...,-2.090744,-7.135138,-3.434698,-0.795288,3.185662,-2.611576,-2.380251,-7.135138,-0.420893,1.126957
522,PP-65002,PP-65002,1.196024,3.891377,3.174650,2.052378,-6.061364,1.157804,2.768359,-3.739436,...,-1.537802,-6.061364,-2.360924,-0.775962,4.472939,-3.254009,-2.360924,-6.061364,-0.333444,0.313675
523,PP-65003,PP-65003,0.260479,2.779444,1.707703,1.763393,-1.927148,0.552542,2.269249,-2.464805,...,-2.895439,-5.834039,-3.026684,-1.115220,3.123063,-3.026684,-3.512110,-3.512110,0.112380,0.232051
524,PP-65004,PP-65004,0.821093,2.343860,2.134473,1.670442,-7.067651,-0.247472,1.601234,-4.260296,...,-2.675333,-7.067651,-3.367211,-1.185008,2.961637,-3.160760,-3.160760,-7.067651,-0.056423,0.625836


In [51]:
#make case_id_df
case_id_df = base_tr_case.copy()
case_id_df.drop(case_id_df.iloc[:,2:219245], axis=1, inplace=True)
case_id = case_id_df['FID'].tolist()
print('Case ID: \n', case_id_df.head())

#make control_id_df
cont_id_df = base_tr_cont.copy()
cont_id_df.drop(cont_id_df.iloc[:,2:219245], axis=1, inplace=True)
cont_id = cont_id_df['FID'].tolist()
print('Control ID: \n', cont_id_df.head())

Case ID: 
        FID      IID
0  PP-3001  PP-3001
1  PP-3002  PP-3002
2  PP-3003  PP-3003
3  PP-3006  PP-3006
4  PP-3007  PP-3007
Control ID: 
         FID       IID
0  PP-14426  PP-14426
1  PP-15761  PP-15761
2  PP-17608  PP-17608
3   PP-3000   PP-3000
4   PP-3004   PP-3004


In [53]:
#drop rows covariate data if FID is not in case_id
covar_df_case_scale_trim = covar_df_case_scale[covar_df_case_scale['FID'].isin(case_id)] 
print('Shape of covar_df_case_scale_trim:', covar_df_case_scale_trim.shape)
print(covar_df_case_scale_trim.head())

#drop rows covariate data if FID is not in cont_id
covar_df_cont_scale_trim = covar_df_cont_scale[covar_df_cont_scale['FID'].isin(cont_id)]
print('Shape of covar_df_cont_scale_trim:', covar_df_cont_scale_trim.shape)
print(covar_df_cont_scale_trim.head())

Shape of covar_df_case_scale_trim: (705, 16)
       FID      IID  EDUCATION  FAMILY_HISTORY  MALE       AGE   PC1_gen  \
0  PP-3400  PP-3400   0.692308               0     0  0.098361 -0.004662   
1  PP-3402  PP-3402   0.615385               0     1  0.213115  0.011903   
2  PP-3403  PP-3403   0.692308               0     1  0.590164 -0.014213   
3  PP-3406  PP-3406   0.692308               0     1  0.032787 -0.008313   
4  PP-3407  PP-3407   0.615385               0     1  0.540984  0.033813   

    PC2_gen   PC3_gen   PC4_gen   PC5_gen   PC6_gen   PC7_gen   PC8_gen  \
0 -0.023299  0.006330 -0.001153  0.003980 -0.002586 -0.000425 -0.007179   
1 -0.023524 -0.001647  0.005045 -0.003863 -0.003192 -0.003624  0.000327   
2 -0.025636 -0.002261 -0.003854 -0.017373  0.002293 -0.002750  0.023918   
3 -0.010611  0.017661 -0.010679 -0.007702 -0.004243 -0.004153  0.002486   
4 -0.012696  0.020515 -0.000225 -0.008233 -0.003591  0.002511 -0.004819   

    PC9_gen  PC10_gen  
0 -0.020155 -0.000930  

In [12]:
#save base_aa_case as csv --run only once
base_tr_case.to_hdf(r'./data_folder/base_transcript_case.h5', key='stage', mode='w')
base_tr_cont.to_hdf(r'./data_folder/base_transcript_control.h5', key='stage', mode='w')

#base_ab_case.to_csv(r'./data_folder/base_2_case.csv') --> this takes too long

#save id_df as txt  -- run only once for download
case_id_df.to_csv(r'./data_folder/case_id_df.txt', sep='\t' ,index=False)
cont_id_df.to_csv(r'./data_folder/control_id_df.txt', sep='\t' ,index=False)

#save covariate as txt -- run only once for download
covar_df_case_scale.to_csv(r'./data_folder/te_covariates_case.txt', sep='\t' ,index=False)
covar_df_cont_scale.to_csv(r'./data_folder/te_covariates_control.txt', sep='\t' ,index=False)

Trim plink files with id_df

In [ ]:
#filter sample id using fid_df.txt to reduce rows that are not in fid_df.txt
##plink --bfile <genotype file> --keep <id file> --make-bed --out <outfile>
!plink --bfile /{datadir}/hits_plus_tags_AMPPD_Euros --keep /{datadir}/case_id_df.txt --maf 0.01 --make-bed --out /{datadir}/qc_genotypes_tes_case

In [ ]:
#filter sample id using fid_df.txt to reduce rows that are not in fid_df.txt
##plink --bfile <genotype file> --keep <id file> --make-bed --out <outfile>
!plink --bfile /{datadir}/hits_plus_tags_AMPPD_Euros --keep /{datadir}/control_id_df.txt  --maf 0.01 --make-bed --out /{datadir}/qc_genotypes_tes_control

In [54]:
#check number of IDs in original .fam file
!cat {datadir}/qc_genotypes_tes_case.fam | wc -l
#check .fam file
!head /{datadir}/qc_genotypes_tes_case.fam

#check number of IDs in original .fam file
!cat {datadir}/qc_genotypes_tes_case.bim | wc -l
#check .fam file
!head /{datadir}/qc_genotypes_tes_case.bim

705
PP-3001 PP-3001 0 0 0 -9
PP-3002 PP-3002 0 0 0 -9
PP-3003 PP-3003 0 0 0 -9
PP-3006 PP-3006 0 0 0 -9
PP-3007 PP-3007 0 0 0 -9
PP-3010 PP-3010 0 0 0 -9
PP-3012 PP-3012 0 0 0 -9
PP-3014 PP-3014 0 0 0 -9
PP-3018 PP-3018 0 0 0 -9
PP-3020 PP-3020 0 0 0 -9
305138
1	rs200683566	0	13838	T	C
1	rs370886505	0	14397	C	CTGT
1	rs375086259	0	14653	T	C
1	rs79585140	0	14907	G	A
1	rs199856693	0	14933	A	G
1	rs71252250	0	15118	G	A
1	rs201635489	0	15447	G	A
1	rs201026389	0	16125	G	T
1	rs78588380	0	16257	C	G
1	rs200736374	0	16288	G	C


In [55]:
#check number of IDs in original .fam file
!cat {datadir}/qc_genotypes_tes_control.fam | wc -l
#check .fam file
!head /{datadir}/qc_genotypes_tes_control.fam

#check number of IDs in original .fam file
!cat {datadir}/qc_genotypes_tes_control.bim | wc -l
#check .fam file
!head /{datadir}/qc_genotypes_tes_control.bim

526
PP-3000 PP-3000 0 0 0 -9
PP-3004 PP-3004 0 0 0 -9
PP-3008 PP-3008 0 0 0 -9
PP-3011 PP-3011 0 0 0 -9
PP-3013 PP-3013 0 0 0 -9
PP-3016 PP-3016 0 0 0 -9
PP-3029 PP-3029 0 0 0 -9
PP-3053 PP-3053 0 0 0 -9
PP-3055 PP-3055 0 0 0 -9
PP-3057 PP-3057 0 0 0 -9
305136
1	rs200683566	0	13838	T	C
1	rs370886505	0	14397	C	CTGT
1	rs375086259	0	14653	T	C
1	rs79585140	0	14907	G	A
1	rs199856693	0	14933	A	G
1	rs71252250	0	15118	G	A
1	rs201635489	0	15447	G	A
1	rs201026389	0	16125	G	T
1	rs78588380	0	16257	C	G
1	rs200736374	0	16288	G	C


Compare patient ID in twas and tes (this step is not necessary if you are not comparing)

In [23]:
twas_cont_id = pd.read_csv(f"/data/songy4/twas/data_folder/control_id_df.txt", engine='c', sep='\t')
twas_case_id = pd.read_csv(f"/data/songy4/twas/data_folder/case_id_df.txt", engine='c', sep='\t')
print('number of twas control id:', twas_cont_id.shape)
print('number of twas case id:', twas_case_id.shape)

#make a list on each id columns
twas_case = twas_case_id['FID'].tolist()
twas_cont = twas_cont_id['FID'].tolist()
te_case = case_id_df['FID'].tolist()
te_cont = cont_id_df['FID'].tolist()

number of twas control id: (575, 2)
number of twas case id: (1138, 2)


In [25]:
####compare FID from TE data with TWAS data

def comprehension(a, b):
    return [x for x in a if x in b]

case_id = comprehension(twas_case,te_case)
cont_id = comprehension(twas_cont,te_cont)

print('number of case only patient ID in TWAS and TE: ', len(case_id))
print('number of control only patient ID in TWAS and TE: ', len(cont_id))

number of case only patient ID in TWAS and TE:  401
number of control only patient ID in TWAS and TE:  163


In [26]:
#check if case and control combined lists has more overlap between twas and te
twas_id_list = twas_case + twas_cont
te_id_list = te_case + te_cont
id_list = comprehension(twas_id_list,te_id_list)
print('number of patient ID in TWAS and TE: ', len(id_list))

number of patient ID in TWAS and TE:  568


data statistics (to write a paper)

In [2]:
covariates_case = pd.read_csv(f"{datadir}/te_covariates_case.txt", sep='\t')
covariates_cont = pd.read_csv(f"{datadir}/te_covariates_control.txt", sep='\t')
covariates_case

,FID,IID,EDUCATION,FAMILY_HISTORY,MALE,AGE,PC1_gen,PC2_gen,PC3_gen,PC4_gen,PC5_gen,PC6_gen,PC7_gen,PC8_gen,PC9_gen,PC10_gen
0,PP-3400,PP-3400,0.692308,0,0,0.098361,-0.004662,-0.023299,0.006330,-0.001153,0.003980,-0.002586,-0.000425,-0.007179,-0.020155,-0.000930
1,PP-3402,PP-3402,0.615385,0,1,0.213115,0.011903,-0.023524,-0.001647,0.005045,-0.003863,-0.003192,-0.003624,0.000327,0.012216,-0.006821
2,PP-3403,PP-3403,0.692308,0,1,0.590164,-0.014213,-0.025636,-0.002261,-0.003854,-0.017373,0.002293,-0.002750,0.023918,0.020800,-0.003866
3,PP-3406,PP-3406,0.692308,0,1,0.032787,-0.008313,-0.010611,0.017661,-0.010679,-0.007702,-0.004243,-0.004153,0.002486,0.002382,0.008385
4,PP-3407,PP-3407,0.615385,0,1,0.540984,0.033813,-0.012696,0.020515,-0.000225,-0.008233,-0.003591,0.002511,-0.004819,0.002946,0.009602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,PP-40816,PP-40816,0.692308,1,1,0.622951,0.032421,-0.004262,0.007910,-0.009565,-0.001786,-0.005201,0.002870,-0.002886,0.023377,0.010822
701,PP-41801,PP-41801,0.615385,0,1,0.770492,0.029534,0.000108,-0.000886,0.016254,0.014111,-0.000023,0.008686,0.010129,0.026381,0.001977
702,PP-42079,PP-42079,0.615385,1,1,0.540984,0.034438,-0.004636,-0.039376,-0.014328,-0.000155,-0.008150,-0.008431,-0.021045,-0.001832,-0.005066
703,PP-50027,PP-50027,0.653846,1,1,0.426230,0.033116,0.013657,-0.003440,-0.009161,-0.008775,-0.002881,-0.000913,-0.003348,-0.017472,0.000134


In [3]:
#get percentage of MALE, FAMILY_HISTORY

print('% of MALE column in case (1=MALE, 0=FEMALE): \n', covariates_case['MALE'].value_counts(normalize=True) * 100)
print('% of MALE column in control (1=MALE, 0=FEMALE): \n', covariates_cont['MALE'].value_counts(normalize=True) * 100)

print('% of FAMILY_HISTORY column in case (1=positive, 0=none): \n', covariates_case['FAMILY_HISTORY'].value_counts(normalize=True) * 100)
print('% of FAMILY_HISTORY column in control (1=positive, 0=none): \n', covariates_cont['FAMILY_HISTORY'].value_counts(normalize=True) * 100)

% of MALE column in case (1=MALE, 0=FEMALE): 
 1    57.730496
0    42.269504
Name: MALE, dtype: float64
% of MALE column in control (1=MALE, 0=FEMALE): 
 0    51.520913
1    48.479087
Name: MALE, dtype: float64
% of FAMILY_HISTORY column in case (1=positive, 0=none): 
 0    73.900709
1    26.099291
Name: FAMILY_HISTORY, dtype: float64
% of FAMILY_HISTORY column in control (1=positive, 0=none): 
 1    53.612167
0    46.387833
Name: FAMILY_HISTORY, dtype: float64


In [23]:
#get mean and SD of AGE

print('Mean of AGE column in case: \n', covar_df_case['AGE'].mean())
print('Standard Diviation of AGE column in case: \n', covar_df_case['AGE'].std())


print('Mean of AGE column in control: \n', covar_df_cont['AGE'].mean())
print('Standard Diviation of AGE column in control: \n', covar_df_cont['AGE'].std())

Mean of AGE column in case: 
 63.589358799454295
Standard Diviation of AGE column in case: 
 10.220211746349126
Mean of AGE column in control: 
 58.28971962616822
Standard Diviation of AGE column in control: 
 12.284097501215577
